In [1]:
using Random
using DataFrames
using CSV
using Statistics

struct Fighter
    name::String
    h::Int16
    f::Int16
    l::Int16
    t::Int16
end

const MXS = 10000
const MXV = 100
const MNV = 10

function random_fighter(h0::Int = MNV, f0::Int = MNV, l0::Int = MNV, t0::Int = MNV)::Fighter
    flag = true
    while flag
        h = h0 + rand(0:(MXV - h0), 1)[1]
        f = f0 + rand(0:(MXV - f0), 1)[1]
        l = l0 + rand(0:(MXV - l0), 1)[1]
        t = t0 + rand(0:(MXV - t0), 1)[1]
        mm = minimum([h, f])
        budget = h*f+h*l+f*t
        if budget <= MXS && budget + mm > MXS
            flag = false
            return Fighter("No Name", h, f, l, t)
        end
    end
end

function eval_battle(a::Fighter, b::Fighter)::Int
    a_finds = a.f >= b.h
    b_finds = b.f >= a.h
    if a_finds && b_finds
        if a.t > b.t
            return 1
        end
        if a.t < b.t
            return -1
        end
        if a.t == b.t
            if a.l > b.l
                return 1
            end
            if a.l < b.l
                return -1
            end
            if a.l == b.l
                if a.h > b.h
                    return 1
                end
                if a.h < b.h
                    return -1
                end
                if a.h == b.h
                    return 0
                end
            end
        end
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        if a.l > b.l
            return 1
        end
        if a.l < b.l
            return -1
        end
        if a.l == b.l
            if a.t > b.t
                return 1
            end
            if a.t < b.t
                return -1
            end
            if a.t == b.t
                if a.f > b.f
                    return 1
                end
                if a.f < b.f
                    return -1
                end
                if a.f == b.f
                    return 0
                end
            end
        end
    end
end

function random_tournament_winner(c::Int)::Fighter
    if c==0
        return random_fighter()
    end
    a = random_tournament_winner(c-1)
    b = random_tournament_winner(c-1)
    res = eval_battle(a, b)
    if res == 1
        return a
    else
        return b
    end
end


tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)

function random_name_and_stat()::Fighter
    vp = [0, 0, 0, 0]
    nametype = rand(1:5)
    name = ""
    if nametype == 1 || nametype == 2
        nm = rand(names)
        adj_i = rand(1:nrow(adjectives))
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H]
        vp[2] = vp[2] + adjectives[adj_i, :F]
        vp[3] = vp[3] + adjectives[adj_i, :L]
        vp[4] = vp[4] + adjectives[adj_i, :T]
        if nametype == 1
            name = string(nm, " the ", adj)
        end
        if nametype == 2
            name = string(adj, " ", nm)
        end
    end
    if nametype == 3 || nametype == 4
        nm = rand(names)
        noun_i = rand(1:nrow(nouns))
        noun = nouns[noun_i, :noun]
        vp[1] = vp[1] + nouns[noun_i, :H]
        vp[2] = vp[2] + nouns[noun_i, :F]
        vp[3] = vp[3] + nouns[noun_i, :L]
        vp[4] = vp[4] + nouns[noun_i, :T]    
        if nametype == 3
            name = string(nm, " the ", noun)
        end
        if nametype == 4
            name = string(noun, " ", nm)
        end
    end
    if nametype == 5
        adj_i = rand(1:nrow(adjectives))
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H]
        vp[2] = vp[2] + adjectives[adj_i, :F]
        vp[3] = vp[3] + adjectives[adj_i, :L]
        vp[4] = vp[4] + adjectives[adj_i, :T]
        noun_i = rand(1:nrow(nouns))
        noun = nouns[noun_i, :noun]
        vp[1] = vp[1] + nouns[noun_i, :H]
        vp[2] = vp[2] + nouns[noun_i, :F]
        vp[3] = vp[3] + nouns[noun_i, :L]
        vp[4] = vp[4] + nouns[noun_i, :T]    
        name = string(adj, " ", noun)
    end
    Fighter(name, vp[1], vp[2], vp[3], vp[4])
end

function rand_rename(a::Fighter, n_tries::Int = 10)::Fighter
    best_score = 0.0
    best_b = random_name_and_stat()
    for ii in 1:n_tries
        b = random_name_and_stat()
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.t^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.t * b.t)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Fighter(best_b.name, a.h, a.f, a.l, a.t)
end

function eval_battle_list(a::Fighter, bs::Array{Fighter})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function pick_best(as::Array{Fighter}, bs::Array{Fighter})::Fighter
    bestscore = -999
    bestf = as[1]
    for ii in 1:length(as)
        score = eval_battle_list(as[ii], bs)
        if score > bestscore
            bestscore = score
            bestf = as[ii]
        end
    end
    return bestf
end

function pick_best_rdmly(as::Array{Fighter}, bs::Array{Fighter}, ntries::Int)::Fighter
    bestscore = -999
    bestf = rand(as)
    for ii in 1:ntries
        f = rand(as)
        score = eval_battle_list(f, bs)
        if score > bestscore
            bestscore = score
            bestf = f
        end
    end
    return bestf
end

function fighters_to_df(as::Array{Fighter})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ts = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ts[ii] = as[ii].t    
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, t = ts)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

function eval_team_battle(as::Array{Fighter}, bs::Array{Fighter})::Int
    a_i = 1
    b_i = 1
    while (a_i <= length(as)) && (b_i <= length(bs))
        res = eval_battle(as[a_i], bs[b_i])
        if res == 1
            b_i = b_i + 1
        else
            a_i = a_i + 1
            if res == 0
                b_i = b_i + 1
            end
        end
    end
    a_out = (a_i > length(as))
    b_out = (b_i > length(as))
    if a_out
        if b_out
            return 0
        else
            return -1
        end
    else
        return 1
    end
end

eval_team_battle (generic function with 1 method)

In [2]:
function random_team(f::Function, team_size::Int)::Array{Fighter}
    team = Array{Fighter}(undef, team_size)
    for i in 1:team_size
        team[i] = f()
    end
    return team
end

random_team (generic function with 1 method)

In [3]:
library = Array{Fighter}(undef, 10000)
for i in 1:10000
    library[i] = rand_rename(random_fighter())
end

In [4]:
function evaluate_generator(f::Function, team_size::Int, n_times::Int = 1)::AbstractFloat
    scores = Array{AbstractFloat}(undef, n_times)
    for i0 in 1:n_times
        team = Array{Fighter}(undef, team_size)
        for i in 1:team_size
            team[i] = f()
        end
        bestscore = 999999
        exploiter = library[1]
        for i in 1:length(library)
            fighter = library[i]
            score = 0
            for j in 1:team_size
                score = score + eval_battle(team[j], fighter)
            end
            if score < bestscore
                bestscore = score
                exploiter = library[i]
            end
        end
        if n_times < 5
            println(exploiter)        
        end
        scores[i0] = bestscore/team_size
    end
    return mean(scores)
end

evaluate_generator (generic function with 2 methods)

In [5]:
# new random generation to more closely approximate Nash
function random_scheme_a(budget::Int = MXS, h0::Int = MNV, f0::Int = MNV, l0::Int = MNV, t0::Int = MNV)::Fighter
    budget_sofar = budget + 1
    while budget_sofar > budget
        hf_ratio = rand() * .70 + rand() * 0.03 + 0.02
        l_ratio = rand()
        h_temp = rand() + 0.15
        f_temp = rand() + 0.15
        if hf_ratio > 0.05
            while abs(h_temp - f_temp) < 0.3
                h_temp = rand() + 0.15
                f_temp = rand() + 0.15
            end
        end
        budget_hf = (0.05 + hf_ratio * 0.8) * (budget * .75)
        h_ratio = h_temp/sqrt(h_temp * f_temp)
        f_ratio = f_temp/sqrt(h_temp * f_temp)
        h = max(h0, round(sqrt(budget_hf) * h_ratio))
        f = max(f0, round(sqrt(budget_hf) * f_ratio))
        budget_hf = h * f
        budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
        l_ratio2 = (h^2 + 5 * l_ratio)/(h^2+f^2 + 5)
        #l_ratio2 = (l_ratio * h)/(l_ratio * h + (1-l_ratio) * f)
        lraw = l0 + ( budget_remain * l_ratio2 )/h
        traw = t0 + ( budget_remain * (1 - l_ratio2) )/f
        l = trunc(lraw)
        t = trunc(traw)
        budget_sofar = h * f + h * l + f * t
        if (budget - budget_sofar) >= min(h, f)
            could_add_l = (budget - budget_sofar) >= h
            could_add_t = (budget - budget_sofar) >= f
            if could_add_l && !could_add_t
                l = l+trunc((budget - budget_sofar)/h)
            end
            if !could_add_l && could_add_t
                t = t+trunc((budget - budget_sofar)/f)
            end
            if could_add_l && could_add_t
                if fpart(lraw) > fpart(traw)
                    l = l+trunc((budget - budget_sofar)/h)
                else
                    t = t+trunc((budget - budget_sofar)/f)
                end
            end
        end
    end
    h = convert(Int, h)
    f = convert(Int, f)
    l = convert(Int, l)
    t = convert(Int, t)  
    return Fighter("No Name [Scheme A]", h, f, l, t)
end

#evaluate_generator(random_scheme_a, 1000, 50)

random_scheme_a (generic function with 6 methods)

In [6]:
evaluate_generator(random_scheme_a, 1000, 50)

-0.3618

In [7]:
budget = MXS
h0 = MNV
f0 = MNV
l0 = MNV
t0 = MNV
hf_ratio0 = 0
l_ratio0 = 0
h_temp0 = 0
f_temp0 = 0

hf_ratio = rand()
l_ratio = rand()
h_temp = rand()
f_temp = rand()
budget_hf = (0.1 + hf_ratio * 0.8) * (budget * .7)
h_ratio = h_temp/sqrt(h_temp * f_temp)
f_ratio = f_temp/sqrt(h_temp * f_temp)
h = max(h0, round(sqrt(budget_hf) * h_ratio))
f = max(f0, round(sqrt(budget_hf) * f_ratio))
budget_hf = h * f
budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
l_ratio2 = (h-2)^2/((h-2)^2+(f-2)^2)
#l_ratio2 = (l_ratio * h)/(l_ratio * h + (1-l_ratio) * f)
lraw = l0 + ( budget_remain * l_ratio2 )/h
traw = t0 + ( budget_remain * (1 - l_ratio2) )/f
l = trunc(lraw)
t = trunc(traw)
budget_sofar = h * f + h * l + f * t
if (budget - budget_sofar) >= min(h, f)
    could_add_l = (budget - budget_sofar) >= h
    could_add_t = (budget - budget_sofar) >= f
    if could_add_l && !could_add_t
        l = l+trunc((budget - budget_sofar)/h)
    end
    if !could_add_l && could_add_t
        t = t+trunc((budget - budget_sofar)/f)
    end
    if could_add_l && could_add_t
        if fpart(lraw) > fpart(traw)
            l = l+trunc((budget - budget_sofar)/h)
        else
            t = t+trunc((budget - budget_sofar)/f)
        end
    end
end

println((h, f, l, t, budget_hf, budget_sofar, hf_ratio))

(34.0, 36.0, 121.0, 129.0, 1224.0, 9946.0, 0.09190426136009022)


In [8]:
# budget = MXS
# h0 = MNV
# f0 = MNV
# l0 = MNV
# t0 = MNV
# hf_ratio0 = 0
# l_ratio0 = 0
# h_temp0 = 0
# f_temp0 = 0


# for i in 1:1000
#     hf_ratio = rand()
#     l_ratio = 0.25 + 0.5 * rand()
#     h_temp = rand() + 0.1
#     f_temp = rand() + 0.1
#     budget_hf = sqrt(0.05 + hf_ratio * 0.9) * (budget * .7)
#     h_ratio = h_temp/sqrt(h_temp * f_temp)
#     f_ratio = f_temp/sqrt(h_temp * f_temp)
#     h = max(h0, round(sqrt(budget_hf) * h_ratio))
#     f = max(f0, round(sqrt(budget_hf) * f_ratio))
#     budget_hf = h * f
#     budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
#     lraw = l0 + ( budget_remain * l_ratio )/h
#     traw = t0 + ( budget_remain * (1 - l_ratio) )/f
#     l = trunc(lraw)
#     t = trunc(traw)
#     budget_sofar = h * f + h * l + f * t
#     if (budget - budget_sofar) >= min(h, f)
#         could_add_l = (budget - budget_sofar) >= h
#         could_add_t = (budget - budget_sofar) >= f
#         if could_add_l && !could_add_t
#             l = l+trunc((budget - budget_sofar)/h)
#         end
#         if !could_add_l && could_add_t
#             t = t+trunc((budget - budget_sofar)/f)
#         end
#         if could_add_l && could_add_t
#             if fpart(lraw) > fpart(traw)
#                 l = l+trunc((budget - budget_sofar)/h)
#             else
#                 t = t+trunc((budget - budget_sofar)/f)
#             end
#         end
#     end

#     budget_sofar = h * f + h * l + f * t
#     if (budget - budget_sofar) >= min(h, f)
#         hf_ratio0 = hf_ratio
#         l_ratio0 = l_ratio
#         h_temp0 = h_temp
#         f_temp0 = f_temp
#         println((h, f, l, t, budget_hf, budget_sofar))
#     end
# end

In [9]:
# hf_ratio = hf_ratio0
# l_ratio = l_ratio0
# h_temp = h_temp0
# f_temp = f_temp0

In [10]:
# budget_hf = sqrt(0.05 + hf_ratio * 0.9) * (budget * .7)
# h_ratio = h_temp/sqrt(h_temp * f_temp)
# f_ratio = f_temp/sqrt(h_temp * f_temp)
# h = max(h0, round(sqrt(budget_hf) * h_ratio))
# f = max(f0, round(sqrt(budget_hf) * f_ratio))
# budget_hf = h * f
# budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
# lraw = l0 + ( budget_remain * l_ratio )/h
# traw = t0 + ( budget_remain * (1 - l_ratio) )/f
# l = trunc(lraw)
# t = trunc(traw)
# budget_sofar = h * f + h * l + f * t
# println((h, f, l, t, budget_hf, budget_sofar))

In [11]:
# if (budget - budget_sofar) >= min(h, f)
#     could_add_l = (budget - budget_sofar) >= h
#     could_add_t = (budget - budget_sofar) >= f
#     if could_add_l && !could_add_t
#         l = l+1
#     end
#     if !could_add_l && could_add_t
#         t = t+1
#     end
#     if could_add_l && could_add_t
#         if fpart(lraw) > fpart(traw)
#             l = l+1
#         else
#             t = t+1
#         end
#     end
# end

# budget_sofar = h * f + h * l + f * t
# println((h, f, l, t, budget_hf, budget_sofar))

## Testing Random Scheme A

In [12]:
f = random_scheme_a()
print(f)
f.h * f.f + f.h * f.l + f.f * f.t

Fighter("No Name [Scheme A]", 35, 19, 203, 117)

9993

In [13]:
rt = random_team(() -> rand_rename(random_scheme_a()), 1000)
fighters_to_df(rt)[1:10, :]

Row,name,h,f,l,t
,String,Int64,Int64,Int64,Int64
1,Joravar the Heron,37,87,32,64
2,Invincible Isaia,29,47,85,131
3,Super Lamp,20,69,41,113
4,Powerful Leopard,25,75,40,95
5,Beetle Arbor,93,24,76,29
6,Aggressive Kangaroo,85,33,71,35
7,Egotistical Yoyo,39,78,40,69
8,Ravenous Yoyo,44,91,30,51
9,Dynamic Notebook,48,94,28,44


In [14]:
counters = random_team(() -> pick_best_rdmly(library, rt, 9), 100)
fighters_to_df(counters)[1:10, :]

Row,name,h,f,l,t
,String,Int64,Int64,Int64,Int64
1,Royal Hanan,11,98,62,84
2,Jax the Shiny,11,85,66,98
3,Viggo the Jacket,84,31,77,29
4,Junior Xray,39,57,79,82
5,Brynnlee the Kitten,40,56,98,68
6,Makkah the Old,53,91,80,10
7,Arinze the Penguin,55,88,68,16
8,Seasoned Phoenix,25,94,54,67
9,Joyful Frog,62,35,83,76


In [15]:
for f in rt
    v = eval_battle_list(f, counters)
    if v < -50
        println(f)
        println(v)
    end
end

Fighter("Aggressive Kangaroo", 85, 33, 71, 35)
-56
Fighter("Hatem the Canard", 116, 41, 38, 20)
-52
Fighter("Cricket Attila", 146, 29, 36, 17)
-62
Fighter("Train Elaina", 20, 27, 169, 225)
-52
Fighter("Neurotic Rogue", 86, 41, 59, 34)
-70
Fighter("Ancient Penguin", 75, 44, 64, 43)
-62
Fighter("Xquisite Jaior", 72, 43, 69, 45)
-58
Fighter("Sheltering Mahid", 105, 41, 45, 23)
-52
Fighter("Quiet Koala", 91, 42, 54, 30)
-72
Fighter("Shy Kangaroo", 114, 37, 43, 23)
-62
Fighter("Khade the Cat", 92, 35, 62, 30)
-66
Fighter("Jacket Phoenix", 86, 36, 66, 34)
-68
Fighter("Sphere Levana", 87, 40, 59, 34)
-72
Fighter("Jru the Sheltering", 116, 27, 54, 22)
-60
Fighter("Eshan the Ladder", 16, 40, 87, 199)
-54
Fighter("Azori the Blueberry", 119, 35, 43, 20)
-62
Fighter("Canard Cailin", 125, 36, 38, 20)
-62
Fighter("Violet Jacques", 89, 38, 61, 31)
-68
Fighter("Demirr the Cube", 16, 44, 75, 184)
-60
Fighter("Nerdy Horse", 100, 32, 59, 28)
-56
Fighter("Silent Tyro", 87, 39, 61, 33)
-68
Fighter("Elegant

In [16]:
for f in rt
    v = eval_battle_list(f, counters)
    if v > 30
        println(f)
        println(v)
    end
end

Fighter("Super Lamp", 20, 69, 41, 113)
32
Fighter("Umbrella Coralai", 79, 19, 100, 31)
50
Fighter("Kitten Everitt", 67, 28, 101, 48)
50
Fighter("Ranveer the Merciful", 73, 24, 100, 39)
50
Fighter("Madelyn the Jacket", 83, 17, 97, 31)
46
Fighter("Skya the Dizzy", 80, 18, 100, 31)
52
Fighter("Maliek the Balanced", 62, 24, 117, 52)
46
Fighter("Lennex the Beautiful", 66, 22, 114, 46)
50
Fighter("Kylan the Shy", 76, 20, 102, 36)
52
Fighter("Indigo the Powerful", 20, 76, 36, 102)
34
Fighter("Berel the Violet", 62, 16, 134, 43)
46
Fighter("Ostrich Keontay", 26, 66, 49, 106)
32
Fighter("Unicorn Kayceon", 56, 21, 136, 57)
32
Fighter("Tricky Mallet", 20, 93, 25, 82)
32
Fighter("Heroic Icicle", 23, 84, 33, 87)
34
Fighter("Prankster Eero", 65, 25, 110, 49)
48
Fighter("Zamzam the Warrior", 22, 70, 44, 107)
32
Fighter("Mathematician Advaita", 69, 22, 109, 43)
50
Fighter("Gamer Leeandre", 28, 66, 50, 102)
32
Fighter("Creative Riyansh", 60, 28, 112, 57)
40
Fighter("Orange Carletta", 59, 28, 113, 60)
4

In [17]:
evaluate_generator(random_scheme_a, 100, 20)

-0.4580000000000001

In [18]:
evaluate_generator(random_fighter, 100, 20)

-0.8389999999999999

In [19]:
evaluate_generator(() -> random_tournament_winner(1), 100, 20)

-0.8500000000000002

In [20]:
evaluate_generator(() -> random_tournament_winner(2), 100, 20)

-0.8109999999999999

In [21]:
evaluate_generator(() -> random_tournament_winner(3), 100, 20)

-0.8119999999999999

In [22]:
evaluate_generator(() -> random_tournament_winner(4), 100, 20)

-0.7794999999999999

## Team battles

In [23]:
team_a = random_team(() -> rand_rename(random_scheme_a()), 100)
team_r0 = random_team(() -> rand_rename(random_tournament_winner(0)), 100)
team_r1 = random_team(() -> rand_rename(random_tournament_winner(1)), 100)
team_r2 = random_team(() -> rand_rename(random_tournament_winner(2)), 100)


100-element Vector{Fighter}:
 Fighter("Dizzy Rhylei", 90, 17, 89, 27)
 Fighter("Reegan the Quilt", 57, 70, 89, 13)
 Fighter("Heroic Pangolin", 51, 81, 32, 52)
 Fighter("Violin Haden", 21, 88, 78, 74)
 Fighter("Spider Dreyah", 97, 78, 13, 15)
 Fighter("Classy Manta", 61, 31, 88, 88)
 Fighter("Armadillo Vivie", 99, 41, 38, 53)
 Fighter("Leyna the Monstrous", 26, 76, 39, 92)
 Fighter("Jac the Avocado", 67, 38, 96, 26)
 Fighter("Train Novie", 37, 61, 60, 90)
 Fighter("Super Fish", 66, 55, 49, 57)
 Fighter("Alula the Warrior", 21, 84, 16, 94)
 Fighter("Dragon Diorr", 93, 30, 58, 60)
 ⋮
 Fighter("Energetic Avigayil", 95, 26, 70, 33)
 Fighter("Zookeeper Fay", 50, 87, 71, 24)
 Fighter("Coder Sahal", 86, 60, 47, 13)
 Fighter("Sassy Jacket", 89, 71, 21, 25)
 Fighter("Minimal Meko", 76, 83, 18, 28)
 Fighter("Adekunle the Cricket", 91, 39, 52, 44)
 Fighter("Sander the Mouse", 64, 63, 64, 29)
 Fighter("Silent Umbrella", 80, 56, 42, 38)
 Fighter("Potassium Mouse", 76, 63, 55, 16)
 Fighter("Roshan th

In [24]:
eval_team_battle(team_a, counters)

-1

In [25]:
eval_team_battle(team_a, team_r0)

1

In [26]:
eval_team_battle(team_a, team_r1)

1

In [27]:
eval_team_battle(team_a, team_r2)

-1

In [28]:
eval_team_battle(team_r0, team_a)

-1

In [29]:
eval_team_battle(team_r0, team_r1)

-1

In [30]:
eval_team_battle(team_r0, team_r2)

-1

In [31]:
as = counters
bs = team_a
a_i = 1
b_i = 1
print("         ")
println(as[a_i])
print("         ")
println(bs[b_i])
while (a_i <= length(as)) && (b_i <= length(bs))
    res = eval_battle(as[a_i], bs[b_i])
    if res == 1
        println(string(as[a_i].name, " won against ", bs[b_i].name))
        b_i = b_i + 1
        print("         ")
        println(bs[b_i])
    else
        if res == 0
            println(string(as[a_i].name, " tied against ", bs[b_i].name))
            b_i = b_i + 1
            print("         ")
            println(bs[b_i])
        else
            println(string(as[a_i].name, " lost against ", bs[b_i].name))
        end
        a_i = a_i + 1
        print("         ")
        println(as[a_i])
    end
end
a_out = (a_i > length(as))
b_out = (b_i > length(as))
if a_out
    if b_out
        return 0
    else
        return -1
    end
else
    return 1
end


         Fighter("Royal Hanan", 11, 98, 62, 84)
         Fighter("Moody the Fish", 98, 47, 43, 25)
Royal Hanan won against Moody the Fish
         Fighter("Blaykley the Kitten", 83, 36, 69, 35)
Royal Hanan won against Blaykley the Kitten
         Fighter("Valiant Fireball", 42, 75, 43, 67)
Royal Hanan won against Valiant Fireball
         Fighter("Leonid the Giant", 13, 42, 71, 203)
Royal Hanan lost against Leonid the Giant
         Fighter("Jax the Shiny", 11, 85, 66, 98)
Jax the Shiny lost against Leonid the Giant
         Fighter("Viggo the Jacket", 84, 31, 77, 29)
Viggo the Jacket won against Leonid the Giant
         Fighter("Mantis Leonard", 26, 143, 16, 41)
Viggo the Jacket lost against Mantis Leonard
         Fighter("Junior Xray", 39, 57, 79, 82)
Junior Xray won against Mantis Leonard
         Fighter("Xpanded Poet", 58, 26, 119, 61)
Junior Xray lost against Xpanded Poet
         Fighter("Brynnlee the Kitten", 40, 56, 98, 68)
Brynnlee the Kitten lost against Xpanded Poet
     

         Fighter("Heroic Mouse", 51, 94, 26, 41)
Militant Artist lost against Heroic Mouse
         Fighter("Triangle Tildyn", 58, 47, 52, 90)
Triangle Tildyn lost against Heroic Mouse
         Fighter("Dog Aazil", 41, 60, 81, 70)
Dog Aazil won against Heroic Mouse
         Fighter("Alyse the Giraffe", 25, 165, 17, 33)
Dog Aazil won against Alyse the Giraffe
         Fighter("Gallant Fireball", 21, 42, 92, 171)
Dog Aazil lost against Gallant Fireball
         Fighter("Junior Cristiana", 99, 13, 84, 30)
Junior Cristiana lost against Gallant Fireball
         Fighter("Tiyana the Dancing", 83, 28, 86, 19)
Tiyana the Dancing won against Gallant Fireball
         Fighter("Slingshot Alliah", 34, 81, 39, 73)
Tiyana the Dancing won against Slingshot Alliah
         Fighter("Pigeon Darleny", 90, 31, 69, 32)
Tiyana the Dancing won against Pigeon Darleny
         Fighter("Poet Fredrik", 82, 18, 97, 31)
Tiyana the Dancing lost against Poet Fredrik
         Fighter("Zookeeper Jawaun", 66, 100, 19, 

LoadError: BoundsError: attempt to access 100-element Vector{Fighter} at index [101]

In [32]:
a_i, b_i

(92, 101)